In [31]:
# imports 
import numpy as np
from math import sqrt 
from pprint import pprint
from numpy import array
from scipy.io import arff
import pandas as pd
from time import time

In [32]:
# load data

df=pd.read_csv('Annthyroid_02_v01.csv')  

del df['id']
del df['Unnamed: 0']
df['outlier'] = df.outlier.apply(lambda label: 1 if label == "'yes'" else 0)

df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att13,att14,att15,att16,att17,att18,att19,att20,att21,outlier
0,0.73,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.00060,0.015,0.120,0.082,0.146,0
1,0.24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.030,0.143,0.133,0.108,0
2,0.47,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00190,0.024,0.102,0.131,0.078,0
3,0.64,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00090,0.017,0.077,0.090,0.085,0
4,0.23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.00025,0.026,0.139,0.090,0.153,0


In [33]:
df.shape

(6802, 22)

In [34]:
# data to values
df = df.values

In [35]:
# Extratrees Deep
import argparse
import sys
import pickle
from sklearn.metrics import accuracy_score
sys.path.insert(0, "lib")
from gcforest.gcforest import GCForest
from gcforest.utils.config_utils import load_json
config = load_json("./examples/Annthyroid.json")
gc = GCForest(config)

In [36]:
# train test 
from sklearn.cross_validation import train_test_split
y = df[:,21]
X = df[:,0:21]

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [37]:
# number of class
len(np.unique(y_train))

2

In [38]:
         # X_enc is the concatenated predict_proba result of ExtraTrees

X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
   

[ 2018-04-26 14:33:14,194][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 21)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 21)],y_test.shape=(2245,)
[ 2018-04-26 14:33:14,196][cascade_classifier.fit_transform] group_dims=[21]
[ 2018-04-26 14:33:14,197][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:33:14,199][cascade_classifier.fit_transform] group_ends=[21]
[ 2018-04-26 14:33:14,200][cascade_classifier.fit_transform] X_train.shape=(4557, 21),X_test.shape=(2245, 21)
[ 2018-04-26 14:33:14,201][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 21), X_cur_test.shape=(2245, 21)
[ 2018-04-26 14:33:14,968][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:33:16,043][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-04-26 14:33:17,181][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [39]:
# predict
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:33:25,350][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-04-26 14:33:25,353][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 14:33:25,354][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-04-26 14:33:25,357][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


Test Accuracy of ExtraTrees = 98.841871 %


In [40]:
# Make data + estimators
X_test_enc = gc.transform(X_test)
X_train_enc = X_train_enc.reshape((X_train_enc.shape[0], -1))
X_test_enc = X_test_enc.reshape((X_test_enc.shape[0], -1))
X_train_origin = X_train.reshape((X_train.shape[0], -1))
X_test_origin = X_test.reshape((X_test.shape[0], -1))
X_train_enc = np.hstack((X_train_origin, X_train_enc))
X_test_enc = np.hstack((X_test_origin, X_test_enc))

[ 2018-04-26 14:33:27,868][cascade_classifier.transform] X_groups_test.shape=[(2245, 21)]
[ 2018-04-26 14:33:27,870][cascade_classifier.transform] group_dims=[21]
[ 2018-04-26 14:33:27,871][cascade_classifier.transform] X_test.shape=(2245, 21)
[ 2018-04-26 14:33:27,873][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 21)


In [41]:
# New data
X_train=X_train_enc
X_test=X_test_enc 
X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)

[ 2018-04-26 14:33:30,483][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-04-26 14:33:30,486][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 14:33:30,487][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:33:30,489][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 14:33:30,490][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-04-26 14:33:30,493][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-04-26 14:33:31,370][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-04-26 14:33:32,432][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-04-26 14:33:33,587][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

In [42]:
# predict  data + Es
y_pred = gc.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100))

[ 2018-04-26 14:33:41,628][cascade_classifier.transform] X_groups_test.shape=[(2245, 23)]
[ 2018-04-26 14:33:41,630][cascade_classifier.transform] group_dims=[23]
[ 2018-04-26 14:33:41,632][cascade_classifier.transform] X_test.shape=(2245, 23)
[ 2018-04-26 14:33:41,634][cascade_classifier.transform] [layer=0] look_indexs=[0], X_cur_test.shape=(2245, 23)


Test Accuracy of ExtraTrees = 99.109131 %


In [43]:
def update(X_train,X_test):
    # update Estimators
    M = np.delete(X_train, (-1), axis=1)
    X_train= np.delete(M, (-1), axis=1)
    M = np.delete(X_test, (-1), axis=1)
    X_test = np.delete(M, (-1), axis=1)
    return X_train,X_test

def newdata(X_test_enc, X_train_enc):
    # Make Data + Estimators
    X_train_origin = X_train.reshape((X_train.shape[0], -1))
    X_test_origin = X_test.reshape((X_test.shape[0], -1))
    X_train_enc = np.hstack((X_train_origin, X_train_enc))
    X_test_enc = np.hstack((X_test_origin, X_test_enc))
    return X_test_enc, X_train_enc

def affiche():
    # predict
    y_pred = gc.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print("Test Accuracy of ExtraTrees = {:.6f} %".format(acc * 100),i,'fois')

In [44]:
# Deep
for i in range (20):
    
    t0 = time()
    X_train,X_test=update(X_train,X_test)
    X_test_enc, X_train_enc=newdata(X_test_enc, X_train_enc)
    
    X_train=X_train_enc
    X_test=X_test_enc 
    X_train_enc, X_test_enc = gc.fit_transform(X_train, y_train, X_test=X_test, y_test=y_test)
    tt = time() - t0
    affiche()
    print ("Time ",format(round(tt,3))," seconds")   

[ 2018-04-26 14:33:44,247][cascade_classifier.fit_transform] X_groups_train.shape=[(4557, 23)],y_train.shape=(4557,),X_groups_test.shape=[(2245, 23)],y_test.shape=(2245,)
[ 2018-04-26 14:33:44,249][cascade_classifier.fit_transform] group_dims=[23]
[ 2018-04-26 14:33:44,250][cascade_classifier.fit_transform] group_starts=[0]
[ 2018-04-26 14:33:44,251][cascade_classifier.fit_transform] group_ends=[23]
[ 2018-04-26 14:33:44,252][cascade_classifier.fit_transform] X_train.shape=(4557, 23),X_test.shape=(2245, 23)
[ 2018-04-26 14:33:44,254][cascade_classifier.fit_transform] [layer=0] look_indexs=[0], X_cur_train.shape=(4557, 23), X_cur_test.shape=(2245, 23)
[ 2018-04-26 14:33:45,064][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-04-26 14:33:46,139][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-04-26 14:33:47,206][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 -

('Test Accuracy of ExtraTrees = 99.109131 %', 0, 'fois')
('Time ', '10.651', ' seconds')


[ 2018-04-26 14:33:58,185][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.68%
[ 2018-04-26 14:33:59,280][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.56%
[ 2018-04-26 14:34:00,294][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.34%
[ 2018-04-26 14:34:01,477][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.12%
[ 2018-04-26 14:34:02,521][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.68%
[ 2018-04-26 14:34:03,592][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.46%
[ 2018-04-26 14:34:04,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.68%
[ 2018-04-26 14:34:05,578][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.56%


('Test Accuracy of ExtraTrees = 98.975501 %', 1, 'fois')
('Time ', '10.65', ' seconds')


[ 2018-04-26 14:34:11,376][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-04-26 14:34:12,403][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.34%
[ 2018-04-26 14:34:13,530][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.34%
[ 2018-04-26 14:34:14,655][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=99.12%
[ 2018-04-26 14:34:15,813][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.68%
[ 2018-04-26 14:34:16,951][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.25%
[ 2018-04-26 14:34:18,045][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.90%
[ 2018-04-26 14:34:19,181][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=99.12%


...

('Test Accuracy of ExtraTrees = 99.287305 %', 17, 'fois')
('Time ', '11.013', ' seconds')


[ 2018-04-26 14:37:48,090][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.46%
[ 2018-04-26 14:37:49,140][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=99.12%
[ 2018-04-26 14:37:50,292][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.12%
[ 2018-04-26 14:37:51,372][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.68%
[ 2018-04-26 14:37:52,458][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.68%
[ 2018-04-26 14:37:53,539][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.46%
[ 2018-04-26 14:37:54,505][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.90%
[ 2018-04-26 14:37:55,527][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.46%


('Test Accuracy of ExtraTrees = 99.287305 %', 18, 'fois')
('Time ', '10.668', ' seconds')


[ 2018-04-26 14:38:01,440][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_0.predict)=98.25%
[ 2018-04-26 14:38:02,559][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_1.predict)=98.90%
[ 2018-04-26 14:38:03,628][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_2.predict)=99.12%
[ 2018-04-26 14:38:04,705][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_3.predict)=98.68%
[ 2018-04-26 14:38:05,829][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_4.predict)=98.46%
[ 2018-04-26 14:38:07,118][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_5.predict)=98.90%
[ 2018-04-26 14:38:08,232][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_6.predict)=98.68%
[ 2018-04-26 14:38:09,350][kfold_wrapper.log_eval_metrics] Accuracy(layer_0 - estimator_0 - 10_folds.train_7.predict)=98.68%


('Test Accuracy of ExtraTrees = 99.198218 %', 19, 'fois')
('Time ', '11.459', ' seconds')


# ('Test Accuracy of ExtraTrees = 99.287305 %', 2, 'fois')

# ('Test Accuracy of ExtraTrees = 99.376392 %', 15, 'fois')
('Time ', '10.264', ' seconds')